In [1]:
import spacy
from spacy import displacy
from spacy_lookup import Entity # use for dict NER
import pandas as pd
import glob
from importlib import reload

#### Ingredients

In [2]:
diary = ['butter','buttermilk','butterfat','casein','cheese','cream','creamer',
         'creme','curd','custard','dairy','dulce','egg','eggnog','gelato',
         'ghee','ice cream','kefir','milk','paneer','soft serve','sour cream',
         'whey','whey protein','whipped cream','whites','yogurt','yolks',
         'mayonnaise','mayo']

fruits = ['acai','apple','apples','apricot','apricots','banana','bananas',
          'berries','berry','blackberries','blackberry','blackcurrant',
          'blueberries','blueberry','cantaloupe','cherries','cherry',
          'clementine','clementines','coconut','coconuts','cranberries',
          'cranberry','currant','currants','date','dates','dragonfruit',
          'fig','figs','fruit','fruits','grape','grapefruit','grapes','guava',
          'honeydew','kiwi','kiwis','lemon','lemons','lime','limes','lychee',
          'mandarine','mandarines','mango','mangos','melon','melons',
          'nectarine','nectarines','olive','olives','orange','oranges',
          'papaya','papayas','passionfruit','peach','peaches','pear','pears',
          'pineapple','plum','plums','pomegranate','pomelo','pomelos','prune',
          'prunes','raisin','raisins','raspberries','raspberry','strawberries',
          'strawberry','tangerine','tangerines','watermelon']

grains = ['angel hair','baking powder','baking soda','barley','bread',
          'buckwheat','bulgur','bun','buns','campanelle','cappelletti',
          'cereal','cornmeal','couscous','crackers','dough','fettuccine',
          'flaxseed','flour','fusilli','grain','grano','grits','lasagna',
          'lasagne','linguine','loaf','macaroni','manicotti','millet','muesli',
          'muffins','muffins','noodle','noodles','oatmeal','oats','orzo',
          'pappardelle','pasta','penne','pita','popcorn','pretzel','pretzels',
          'quinoa','ravioli','rice','rigate','rigatoni','rocchetti','rolls',
          'rotini','rye','seitan','shells','spaghetti','spelt','tortellini',
          'tortilla','tortillas','vermicelli','yeast']

protein = ['abalone','anchovy','arctic char','bacon','barracuda','barramundi',
        'bass','bear','beef','beef heart','beef liver','beef tongue','bib',
        'bison','black drum','bluefish','bluenose','bone','breast','brisket',
        'buffalo','bullhead','butterfish','calf liver','calimari','capelin',
        'caribou','carp','catfish','caviar','char','chicken','chicken liver',
        'chilean seabass','chub','chuck','clam','cobia','cockle','cod','coho',
        'coley','conch','cornish game hen','corvina','crab','crab brown',
        'crawfish','crayfish','croaker','cusk','cuttle fish','dab','drum',
        'duck','duck liver','eel','emu','escargot','escolar','filet','fish',
        'flank','flounder','frog','gilthead','gizzards','goat','goose',
        'goose liver','greater amberjack','grenadier','grouper','grouse',
        'guinea hen','haddock','hake','halfmoon fish','halibut','ham',
        'harvest fish','herring','horse','imitation crab','jellyfish',
        'kangaroo','lamb','limpet slipper','lingcod','liver','lobster','loin',
        'mackerel','mahi-mahi','manila','marlin','marrow soup','meat',
        'monkfish','moose','mullet','muskellunge','mussel','mussels','mutton',
        'ocean pout','octopus','opah','opaleye','opossum','orange roughy',
        'organ meats','organs','ostrich','oyster','pangasius','parrotfish',
        'partridge','perch','pheasant','pickerel','pike','pilchards','plaice',
        'pollock','pollock, atlantic','pomfret','pompano','porgy','pork',
        'poultry','pouting','prawn','protein','pumpkinseed','quail','rabbit',
        'red snapper','rib','rockfish','rosefish','round','sablefish','salmon',
        'sanddabs','sardine','scad','scallops','scrod','scup','sea beam',
        'sea turtle','sea urchin','seabass','seafood','seatrout','shad',
        'shank','shark','sheepshead','shrimp','sirloin','skate','snail',
        'snake','sockeye','sole','spearfish','squab','squid','squirrel',
        'striped bass','sturgeon','sucker','sunfish','swai','sweetbreads',
        'swordfish','tatoaba','tenderloin','thigh','tilapia','tilefish',
        'tripe','trout','tuna','turbot','turkey','turtle','veal','venison',
        'wahoo','walleye','weakfish','whitefish','whiting','wing','wolfish',
        'wreckfish','yellowtail'] 

vegetables = ['artichoke','arugula','asparagus','avocado','bean','bean sprouts',
              'beans','beet','beetroot','beets','bell pepper','bell peppers',
              'bok choy','broccoli','brussels sprouts','butter beans',
              'butternut squash','cabbage','carrot','cauliflower','ceci beans',
              'celeriac','celery','chamomile','chard','chickpea','chickpeas',
              'chilis','collard greens','corn','cucumber','cucumbers','dill',
              'eggplant','fennel','garlic','greens','kale','leek','leeks',
              'legumes','lentils','lettuce','mushroom','mushrooms','onion',
              'peas','peppers','potato','potatoes','pumpkin','radicchio',
              'radish','radishes','scallion','scallions','shallot','shallots',
              'snap peas','spinach','sprouts','squash','tomato','tomatoes',
              'turnips','vegetable','yam','zucchini',] 

other = ['acorn','aioli','allspice','almond','almonds','anise','balm','basil',
         'bean','beans','bourbon','broth','capers','capsicum','cardamom',
         'cashew','cashews','chestnuts','chia','chicory','chimichurri','chives',
         'chocolate','chutney','cilantro','cinnamon','clove','cloves',
         'condiment','condiments','coriander','cumin','dijon','dill','dip',
         'dressing','drizzle','fats','flower','frosting','gin','ginger',
         'grease','guacamole','gum','habanero','hazelnut','herb','herbs',
         'hoisin','honey','horseradish','hummus','jalapeno','jam','jasmine',
         'jelly','ketchup','lard','lavender','leaf','lemongrass','licorice',
         'margarine','marigold','marmite','mayo','mint','miso',
         'mix','mole','mustard','myrtle','nut','nutella','nutmeg','nuts','oil',
         'oils','oregano','paprika','parsley','peanut','peanuts','pecan',
         'pecans','pepper','peppermint','pesto','pickle','pickles',
         'pico de gallo','pistachio','pistachios','poppy','powder','preserves',
         'reduction','relish','root','rosemary','rum','safflower','saffron',
         'sage','salsa','salt','sassafras','sauce','sauerkraut','seasoning',
         'seasonings','seed','seeds','sesame','shiso','shortening','spearmint',
         'spice','spices','spread','sriracha','sugar','sunflower','syrup',
         'tabasco','tahini','tarragon','tartar','thyme','tomato sauce',
         'turmeric','tzatziki','tzatzki','vanilla','vegemite','vinegar',
         'vodka','walnut','walnuts','wasabi','water','whisky','wine',
         'wintergreen','yeast','zest','ingredient','ingredients','lemon juice',
         'salad'] 

In [3]:
print((diary))
print((fruits))
print((grains))
print((protein))
print((vegetables))
print((other))

['butter', 'buttermilk', 'butterfat', 'casein', 'cheese', 'cream', 'creamer', 'creme', 'curd', 'custard', 'dairy', 'dulce', 'egg', 'eggnog', 'gelato', 'ghee', 'ice cream', 'kefir', 'milk', 'paneer', 'soft serve', 'sour cream', 'whey', 'whey protein', 'whipped cream', 'whites', 'yogurt', 'yolks', 'mayonnaise', 'mayo']
['acai', 'apple', 'apples', 'apricot', 'apricots', 'banana', 'bananas', 'berries', 'berry', 'blackberries', 'blackberry', 'blackcurrant', 'blueberries', 'blueberry', 'cantaloupe', 'cherries', 'cherry', 'clementine', 'clementines', 'coconut', 'coconuts', 'cranberries', 'cranberry', 'currant', 'currants', 'date', 'dates', 'dragonfruit', 'fig', 'figs', 'fruit', 'fruits', 'grape', 'grapefruit', 'grapes', 'guava', 'honeydew', 'kiwi', 'kiwis', 'lemon', 'lemons', 'lime', 'limes', 'lychee', 'mandarine', 'mandarines', 'mango', 'mangos', 'melon', 'melons', 'nectarine', 'nectarines', 'olive', 'olives', 'orange', 'oranges', 'papaya', 'papayas', 'passionfruit', 'peach', 'peaches', 'pea

#### Tools

In [4]:
# Preparation Tools & Essentials
pte = ["Chef's Knife",
'Chef Knife',       
'Cutting Board',
'Can Opener',
'Measuring Cups',
'Measuring Spoons',
'Mixing Bowls',
'Colander',
'Vegetable Peeler',
'Potato Masher',
'Whisk',
'Salad Spinner',
'Grater',
'Shears',
'Citrus Juicer',
'Garlic Press',
'Paring Knife',
'Bread Knife',
'Honing Ceramic Rod',
'Sharpening Ceramic Rod',
'Knife Sharpener']

# Cookware & Bakeware
cbware = ['Stainless Steel Skillet',
'Sauté Pan',
'Small Saucepan',
'Medium Saucepan',
'Large Pot',
'Cast Iron Skillet',
'Grill Pan',
'Baking Sheet Pan',
'Casserole Dish',
'Broiler Pan',
'Stockpot']

# Cooking Tools & Kitchen Utensils
ctku = ['Spatula',
'Stirring Spoon',
'Tongs',
'Ladle',
'Oven Mitts',
'Trivet',
'Splatter Guard',
'Thermometer']

# Appliances
appliances = ['Immersion Blender',
             'Kitchen Scale',
             'Blender']

# Kitchen essentials
ke = ['Food Storage Containers',
'Aluminum Foil',
'Parchment Paper',
'Towels',
'Sponges',
'Heavy Sponges',
'Dish Rack',
'Ice Cube Tray',
'Trash Bags',
'Trash Bin']

In [5]:
print(pte)
print(ctku)
print(appliances)
print(ke)

["Chef's Knife", 'Chef Knife', 'Cutting Board', 'Can Opener', 'Measuring Cups', 'Measuring Spoons', 'Mixing Bowls', 'Colander', 'Vegetable Peeler', 'Potato Masher', 'Whisk', 'Salad Spinner', 'Grater', 'Shears', 'Citrus Juicer', 'Garlic Press', 'Paring Knife', 'Bread Knife', 'Honing Ceramic Rod', 'Sharpening Ceramic Rod', 'Knife Sharpener']
['Spatula', 'Stirring Spoon', 'Tongs', 'Ladle', 'Oven Mitts', 'Trivet', 'Splatter Guard', 'Thermometer']
['Immersion Blender', 'Kitchen Scale', 'Blender']
['Food Storage Containers', 'Aluminum Foil', 'Parchment Paper', 'Towels', 'Sponges', 'Heavy Sponges', 'Dish Rack', 'Ice Cube Tray', 'Trash Bags', 'Trash Bin']


#### Actions

In [6]:
actions = ['Arrange',
'Baste',
'Beat',
'Blend',
'Brown',
'Build',
'Bury',
'Carve',
'Check',
'Chop',
'Close',
'Cool',
'Correct',
'Cover',
'Crumple',
'Cut',
'Decorate',
'Discard',
'Divide',
'Drape',
'Drop',
'Dry',
'Film',
'Fold',
'Follow',
'Form',
'Force',
'Glaze',
'Insert',
'Lay',
'Leave',
'Lift',
'Make',
'Melt',
'Mince',
'Mix',
'Moisten',
'Mound',
'Open',
'Pack',
'Paint',
'Pierce',
'Pour',
'Prepare',
'Press',
'Prick',
'Pull',
'Puree',
'Push',
'Quarter',
'Raise',
'Reduce',
'Refresh',
'Reheat',
'Replace',
'Return',
'Ring',
'Roast',
'Roll',
'Salt',
'Saute',
'Scatter',
'Scoop',
'Scrape',
'Scrub',
'Season',
'Separate',
'Set',
'Settle',
'Shave',
'Simmer',
'Skim',
'Slice',
'Slide',
'Slip',
'Slit',
'Smear',
'Soak',
'Spoon',
'Spread',
'Sprinkle',
'Stir',
'Strain',
'Strew',
'Stuff',
'Surround',
'Taste',
'Thin',
'Tie',
'Tilt',
'Tip',
'Top',
'Toss',
'Trim',
'Turn',
'Twist',
'Warm',
'Wilt',
'Wind',
'Wrap']

In [7]:
print(actions)

['Arrange', 'Baste', 'Beat', 'Blend', 'Brown', 'Build', 'Bury', 'Carve', 'Check', 'Chop', 'Close', 'Cool', 'Correct', 'Cover', 'Crumple', 'Cut', 'Decorate', 'Discard', 'Divide', 'Drape', 'Drop', 'Dry', 'Film', 'Fold', 'Follow', 'Form', 'Force', 'Glaze', 'Insert', 'Lay', 'Leave', 'Lift', 'Make', 'Melt', 'Mince', 'Mix', 'Moisten', 'Mound', 'Open', 'Pack', 'Paint', 'Pierce', 'Pour', 'Prepare', 'Press', 'Prick', 'Pull', 'Puree', 'Push', 'Quarter', 'Raise', 'Reduce', 'Refresh', 'Reheat', 'Replace', 'Return', 'Ring', 'Roast', 'Roll', 'Salt', 'Saute', 'Scatter', 'Scoop', 'Scrape', 'Scrub', 'Season', 'Separate', 'Set', 'Settle', 'Shave', 'Simmer', 'Skim', 'Slice', 'Slide', 'Slip', 'Slit', 'Smear', 'Soak', 'Spoon', 'Spread', 'Sprinkle', 'Stir', 'Strain', 'Strew', 'Stuff', 'Surround', 'Taste', 'Thin', 'Tie', 'Tilt', 'Tip', 'Top', 'Toss', 'Trim', 'Turn', 'Twist', 'Warm', 'Wilt', 'Wind', 'Wrap']


In [8]:
ingredients = diary + fruits + grains + vegetables + other
tools = pte + ctku + appliances + ke

In [9]:
nlp = spacy.load('en')

# add ingredients tags
# diary_e = Entity(keywords_list=diary, label='IDiary')
# fruits_e = Entity(keywords_list=fruits, label='IFruit')
# grains_e = Entity(keywords_list=grains, label='IGrain')
# protein_e = Entity(keywords_list=protein, label='IProtein')
# other_e = Entity(keywords_list=other, label='IOther')
# vegetables_e = Entity(keywords_list=vegetables, label='IVeg')
ingredients_e = Entity(keywords_list=ingredients, label='INGRD')

# add tools tags
# pte_e = Entity(keywords_list=pte, label='TPTE')
# ctku_e = Entity(keywords_list=ctku, label='TCTKU')
# appliances_e = Entity(keywords_list=appliances, label='TAPP')
# ke_e = Entity(keywords_list=ke, label='TKE')
tools_e = Entity(keywords_list=tools, label='TOOL')

# add actions tags
actions_e = Entity(keywords_list=actions, label='ACTION')

# add to pipeline
# nlp.add_pipe(actions_e, last=True, name="Action")
# nlp.add_pipe(diary_e, last=True, name="IDiary")
# nlp.add_pipe(fruits_e, last=True, name="IFruit")
# nlp.add_pipe(grains_e, last=True, name="IGrain")
# nlp.add_pipe(protein_e, last=True, name="IProtein")
# nlp.add_pipe(vegetables_e, last=True, name="IVeg")
# nlp.add_pipe(other_e, last=True, name="IOther")
# nlp.add_pipe(pte_e, last=True, name="TPte")
# nlp.add_pipe(ctku_e, last=True, name="TCtku")
# nlp.add_pipe(appliances_e, last=True, name="TApp")
# nlp.add_pipe(ke_e, last=True, name="TKe")
nlp.add_pipe(actions_e, last=True, name="Action")
nlp.add_pipe(ingredients_e, last=True, name="Ingredient")
nlp.add_pipe(tools_e, last=True, name="Tool")


for name in nlp.pipeline:
    print(name)

('tagger', <spacy.pipeline.Tagger object at 0x7f00ee4c8518>)
('parser', <spacy.pipeline.DependencyParser object at 0x7f00ee47d938>)
('ner', <spacy.pipeline.EntityRecognizer object at 0x7f00ee47d990>)
('Action', <spacy_lookup.Entity object at 0x7f00f0301128>)
('Ingredient', <spacy_lookup.Entity object at 0x7f00f0301080>)
('Tool', <spacy_lookup.Entity object at 0x7f00f0301198>)


In [ ]:
# example sentence to do NER
# doc = nlp(u"Spoon out the meat from the papaya and add them to the mixing bowls.")

# displacy.render(doc, style='ent', jupyter=True)

In [ ]:
# for i in range(len(doc.ents)):
#     if doc.ents[i].label_ not in exclude_these:
#         print(doc.ents[i], doc.ents[i].start_char, doc.ents[i].end_char, doc.ents[i].label_)

In [ ]:
# {"text": "Spoon out the meat from the papaya and add them to the mixing bowls.", 
#  "spans": [
#      {"start": 0, "end": 5, "label": "NAME"}
#      {"start": 14, "end": 18 , "label": "NAME"}
#      {"start": 28, "end": 34, "label": "NAME"}
#      {"start": 55, "end": 67, "label": "NAME"}
#  ]}

#path to folders
#save_dir = "./save/"

#create jsonl file for prodigy:
# jsonl = open("test2.jsonl", "w")
# line = {}
# line['text'] = doc.text

# entities = []
# for i in range(len(doc.ents)):
    
#     entities.append({"start" : doc.ents[i].start_char,
#                      "end" : doc.ents[i].end_char,
#                      'label': doc.ents[i].label_ 
#                     })
# line['spans'] = entities
# jsonl.writelines(json.dumps(line) + "\n")
# jsonl.close()

In [10]:
# bring 1 json into a dataframe for analysis
files = sorted(glob.glob('layer1.json.[0].valid.gz'))
df = pd.concat([pd.read_pickle(fp) for fp in files], ignore_index=True)
df.shape

(99999, 9)

In [11]:
# for simplicity, let's filter out epicurious recipes with less than 5 steps
keep_columns = ['id', 'partition', 
                'collection', 'title', 
                'ingr_count', 'instr_count', 
                'ingredients', 'instructions']

epicurious = df[(df.collection == 'www.epicurious.com') 
   & (df.instr_count<=5)][keep_columns].sort_values(by=['instr_count'], ascending=False)

# train, val, test counts
epicurious.groupby('partition').size()

partition
test     111
train    489
val      110
dtype: int64

In [12]:
train_recipes = epicurious[(epicurious.partition=='train')].index
len(train_recipes)

489

In [ ]:
# Garlic Broth with Sage and Parsley, 21658
# Grilled Corn on the Cob, 96647

# for recipe in test_recipes:
#     # title
#     print('\n' + epicurious['title'][recipe] + '\n')
    
#     # ingredients
#     for index, each in enumerate(epicurious['ingredients'][recipe]):
#         print('Ingredient', index+1, ':', list(each.values())[0])
    
#     # instructoins    
#     for index, each in enumerate(epicurious['instructions'][recipe]):
#         print('Step', index+1, ':', list(each.values())[0])

In [13]:
# spaCy parameters under Token:
# Text: The original word text.
# Lemma: The base form of the word.
# POS: The simple part-of-speech tag.
# Tag: The detailed part-of-speech tag.
# Dep: Syntactic dependency, i.e. the relation between tokens.
# Shape: The word shape – capitalisation, punctuation, digits.
# is alpha: Is the token an alpha character?
# is stop: Is the token part of a stop list, i.e. the most common words of the language?

for recipe in train_recipes[0:10]:
    # title
    print('\n' + epicurious['title'][recipe].upper() + '\n')

    # ingredients    
#     for each in epicurious['ingredients'][recipe]:
#         ingredient = list(each.values())[0]
#         displacy.render(nlp(ingredient), style='ent', jupyter=True)
    
    # instructoins    
    for each in epicurious['instructions'][recipe]:
        instruction = list(each.values())[0]
        displacy.render(nlp(instruction), style='ent', jupyter=True)
        # print parser tree for each step
        #explacy.print_parse_info(nlp, instruction)


CONFETTI SALAD WITH RANCH DRESSING



/opt/conda/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)



RANCH DRESSING



/opt/conda/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)



BUBBAS BEER BISCUITS



...


APPLE CIDER PUNCH



/opt/conda/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)



MANGO JICAMA CHOPPED SALAD




GRILLED PEACHES WITH PECORINO AND PROSCIUTTO



/opt/conda/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [ ]:
# for each in (epicurious['ingredients'].head(1)):
#     for k, v in enumerate(each):
#         doc = nlp(list(v.values())[0])
#         print(list(v.values())[0])
#         for chunk in doc.noun_chunks:
#             print(chunk.root)

In [ ]:
# import benepar
# benepar.download('benepar_en')

In [ ]:
# from benepar.spacy_plugin import BeneparComponent

In [ ]:
# nlp = spacy.load('en')
# nlp.add_pipe(BeneparComponent('benepar_en'))

In [ ]:
# doc = nlp("Bring some light chicken broth to a boil with a few leaves of fresh sage.")
# sent = list(doc.sents)[0]

In [ ]:
# for each in enumerate (sent._.children):
#     print(each)

In [ ]:
# type(sent._.children)

In [ ]:
# for each in enumerate (sent._.children):
#     # go from the start to the end of each span, returning each token in the sentence
#     # combine each token using join()
#     print(each.parse_string)

In [14]:
test_all_recipes = epicurious[(epicurious.partition=='test')].index

test_recipes = test_all_recipes[0:50]

len(test_recipes)

50

In [15]:
import json
exclude_these = [
'PERSON',
'NORP',
'FAC',
'ORG',
'GPE',
'LOC',
'PRODUCT',
'EVENT',
'WORK_OF_ART',
'LAW',
'LANGUAGE',
'MONEY',
'ORDINAL', 
'DATE',
'PERCENT',    
'CARDINAL'    
]

nlp = spacy.load('en')

ingredients_e = Entity(keywords_list=ingredients, label='INGRD')
tools_e = Entity(keywords_list=tools, label='TOOL')
actions_e = Entity(keywords_list=actions, label='ACTION')

nlp.add_pipe(actions_e, last=True, name="Action")
nlp.add_pipe(ingredients_e, last=True, name="Ingredient")
nlp.add_pipe(tools_e, last=True, name="Tool")

jsonl = open("test_100_KH.jsonl", "w")
line = {}

counter = 0
for recipe in test_recipes: 
    for each in epicurious['instructions'][recipe]:
        for instruction in list(each.values()):
            if counter <100:
                doc = nlp(instruction)
                
                line['text'] = doc.text
                entities = []
                
                for i in range(len(doc.ents)):
                    if doc.ents[i].label_ not in exclude_these:
                        entities.append({"start" : doc.ents[i].start_char,
                                         "end" : doc.ents[i].end_char,
                                         'label': doc.ents[i].label_ 
                                        })
                line['spans'] = entities
                line['answer'] = 'reject'
                
                jsonl.writelines(json.dumps(line) + "\n")
                counter = counter+1
jsonl.close()

In [ ]:
# from prodigy.components.db import connect
# db = connect()
#dataset = db.get_dataset('test_100_KH')